In [15]:
import interfaces as inter
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk
from nltk.corpus import wordnet
import numpy as np
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /home/nikita/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/nikita/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
file = 'war-and-peace.txt'
inter.file_to_n_files(file,4)

In [13]:
#t = %timeit -o !mpiexec —allow-run-as-root -n 4 python main.py
#will not work now, functions should be implemented first!
times = []
for n_use in [1000]:
    for n_keep in [10]:
        for n_final_top in [5]:
            f = open('main.py','w')
            f.write('''import interfaces as inter
from mpi4py import MPI
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk
from nltk.corpus import wordnet
wordnet_lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

N_USE = n_use
N_KEEP = n_keep
N_FINAL_TOP = n_final_top
FILENAME = 'war-and-peace'

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
filename = FILENAME + '_0' + str(rank) + '.txt'

inter.pipeline(filename, N_USE, N_KEEP, rank, comm,N_FINAL_TOP)'''.replace('n_use',str(n_use)).replace('n_keep',str(n_keep)).replace('n_final_top',str(n_final_top)))
            f.close()
            t = %timeit -o !mpiexec -n 4 python main.py
            times.append(t)



4.07 s ± 134 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
np.load('result/words_0.txt',allow_pickle=True)

array(['say', 'one', 'go', 'prince', 'pierre'], dtype='<U6')

In [18]:
np.load('result/counts_0.txt',allow_pickle=True)

array([3842, 2192, 2123, 1804, 1616])